# Lecture 8 of Tools For Computational Biology

### Types

#### Integers

In [1]:
x = 1

type(x)

int

#### Floats

In [2]:
x = 3.7

type(x)

float

#### Lists

In [3]:
x = [1, 2, 3]

len(x)

print(x)

[1, 2, 3]


#### Tuples
Tuples differ from lists in that they are immutable, while lists are mutable.

In [4]:
tup = (1, 2, 3)

print(tup)

(1, 2, 3)


#### Dictionaries
Dictionaries are "look up tables."

In [5]:
rc_dict = {'A':'T', 'T':'A', 'C':'G', 'G':'C'}

rc_dict

{'A': 'T', 'T': 'A', 'C': 'G', 'G': 'C'}

You can look up the value for a key:

In [6]:
rc_dict['A']

'T'

You can get a list of all of the keys:

In [7]:
list(rc_dict.keys())

['A', 'T', 'C', 'G']

You can do the same for the values:

In [8]:
list(rc_dict.values())

['T', 'A', 'G', 'C']

### Mutable versus immutable types
An immutable type means that each variable has a fixed value, and you can't change that value without essentially destroying the original variable and creating a new one:

In [9]:
x = 1
y = x
y = 2

What is the value of `x`?

An integer, is an immutable type.
So `y` is assigned the value of `x`, but it's not the same thing as x:

In [10]:
print(x)
print(y)

1
2


Things behave differently with mutable types.
So a list is mutable.
This code defines `x` and then changes its value:

In [11]:
x = [1, 2]
x.append(3)
print(x)

[1, 2, 3]


In [12]:
y = x
print(x)
print(y)

[1, 2, 3]
[1, 2, 3]


Now we are going to change `y`:

In [13]:
y.append(4)
print(y)

[1, 2, 3, 4]


What is `x`?

In [14]:
print(x)

[1, 2, 3, 4]


The difference between a tuple and a list, is that a list is mutable and tuple is not:

In [15]:
xtuple = (1, 2)
xlist = [1, 2]
xlist[0] = 500
print(xlist)

# can't do this
xtuple[0] = 500

[500, 2]


TypeError: 'tuple' object does not support item assignment

A tuple cannot be modified (so use it if what you created is going to never change), whereas a list can be modified.

In dictionaries, you can only use immutable types as the keys.

## Built-in modules
One of the things that makes Python really powerful, is that there are a lot of extra modules and packages.

Some are "built-in", which means that you get them if you install Python.

First we'll look at built-in modules.

### math
The math module has a lot of basic functions that involve **math**.

In [16]:
# import the math module
import math

# dot means use that function from the math module
math.sqrt(2)

1.4142135623730951

You can import functions so that you don't need to add the module name:

In [17]:
from math import *

sqrt(2)

1.4142135623730951

Do **not** do this unless you have a good reason.
The confusing part is that now someone looking at your code doesn't know what the function does unless they know that you imported `math`.

In general, the potential confusion outweights the benefits.

### os and glob
Two that are very useful are `os` and `glob` (also `shutil`, which I won't describe here).

These are for doing shell-like commands.
Basically, unless you are doing something really simple or really computatinally intensive, there is no good reason to program in shell.
It will probably be a lot simpler for you to write and others to read if you use Python and modules like `os` and `glob`.

In [18]:
import os

os.listdir('.')

['barcodes_R1.fastq', 'README.md', 'lecture_8.ipynb', '.ipynb_checkpoints']

In shell, you can use `*` as a wildcard. You can also do this in `glob`:

In [19]:
import glob
glob.glob('*.ipynb')

['lecture_8.ipynb']

### re
The `re` module is for "regular expressions". 
These are very useful for parsing strings.

Here is a common example from my work with influenza.
You download some strains from the database, and they have a name that looks like this:

In [20]:
strain1 = 'A/New York/3/1994 (H3N2)'
strain2 = 'A/California/3/X/2003 (H12N1)'
strain3 = 'A/Perth/2009 (H3N2)'

strains = [strain1, strain2, strain3]

You want to get some information out of these, like the year.

Regular expressions allow you to define patterns in a string:

In [21]:
import re

strainmatch = re.compile(
        '(?P<year>\d{4}) \((?P<subtype>H\d+N\d+)\)$')

subtype_counter = {}
for strain in strains:
    m = strainmatch.search(strain)
    year = m.group('year')
    subtype = m.group('subtype')
    if subtype in subtype_counter:
        subtype_counter[subtype] += 1
    else:
        subtype_counter[subtype] = 1
        
print(subtype_counter)

{'H3N2': 2, 'H12N1': 1}


## External packages
There are also packages, that do **not** come with your Python, and that you have to install separately.

These other packages are on something called `PyPI`.

### Installing with pip
If you needed to install biopython, you would do:

    pip install biopython
    
In general, you install Python packages with `pip`.
If you did not install Python with `conda`, there are all these complex things about permissions, but we are not going to go into that.

This just works, because you all have BioPython installed:

In [22]:
import Bio
print(Bio.__version__)

1.72


This probably will not work for you unless you first went to the terminal and types:

    pip install regex

In [23]:
import regex

## Biopython
Biopython has a lot of useful biology tools in there.
It is not very fast, but it is great it reading in files with lots of different formats.

So the `Bio.SeqIO.parse` command is your friend.

## Practical example
Parsing barcodes.

The sequences are as follows:

    5'-[end of HA]-AGGCGGCCGC-[16 X N]-3'
    
or 

    5'-[end of NA]-AGGCGGCCGC-[16 X N]-3'
    
The end of NA is:

    TGGCCTGATGGAGCAGAACTCCCTTTCACGATAGATAAATAATAGTGCACCAT
    
The end of HA is:

    ATGTGTTCCAACGGCTCCTTGCAATGCCGGATTTGCATATAATGATGCACCAT
    
In our example, I've included a FASTQ file with the barcodes:

In [24]:
glob.glob('*.fastq')

['barcodes_R1.fastq']

We are going to read the sequences into a list:

In [25]:
import Bio.SeqIO

reads_records = list(Bio.SeqIO.parse(
        'barcodes_R1.fastq', 'fastq'))

print(reads_records[0])

ID: HISEQ:621:HMJGNBCX2:1:1101:1797:2150
Name: HISEQ:621:HMJGNBCX2:1:1101:1797:2150
Description: HISEQ:621:HMJGNBCX2:1:1101:1797:2150 1:N:0:ATGTCA
Number of features: 0
Per letter annotation for: phred_quality
Seq('GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGA...AGA', SingleLetterAlphabet())


Often, we want to simplify what we read from BioPython to just be a string:

In [26]:
reads = []
for r in reads_records:
    reads.append(str(r.seq))
    
reads[0]

'GCTTAAGTTATTTAGTGCGGCCGCCTATGGTGCACTATTATTTATCTATCGTGAAAGGGAGTTCTGCTCCATCAGGCCAAGATCGGAAGAGCACACGTCTGAACTCCAGTCACATGTCAGAATCTCGTATGCCGTCTTCTGCTTGAAAAAAAAAAAAAAGCTGAAATTAATAATTTTGAAACCAGTTTTGTAAACGCAGCACTAAAATGAAGGCATGCAACGACGATGTTTATTGACACGGAATAGCAGA'

We want to get barcodes out of strings.
First, we need to reverse complement the reads:

In [27]:
def reverseComplement(seq):
    """Gets the reverse complement of `seq`."""
    rc_dict = {'A':'T', 'T':'A', 
               'C':'G', 'G':'C',
               'N':'N'}
    rc = []
    for nt in seq:
        rc.append(rc_dict[nt])
    rc = reversed(rc)
    rc = ''.join(rc) # turn into string
    return rc

In [28]:
reads_rc = []
for read in reads:
    reads_rc.append(reverseComplement(read))

Get the barcodes. 
So, we know from above that the barcodes are the first 16 nucleotides in each read.

We also know that they next thing should be the reverse complement of this sequence `AGGCGGCCGC`:

Make a regular expression that requires that sequence right after the barcode:

In [29]:
barcode_match = re.compile(
        'AGGCGGCCGC(?P<barcode>[ACGT]{16})$')

barcodes = []

for read in reads_rc:
    barcode = barcode_match.search(read)
    if barcode:
        barcodes.append(barcode.group(
                'barcode'))
        
print(f"Found {len(barcodes)} barcodes in "
      f"{len(reads)} reads.")

Found 9431 barcodes in 10000 reads.


We count the barcodes using a dictionary:

In [30]:
barcode_counts = {}

for barcode in barcodes:
    if barcode in barcode_counts:
        barcode_counts[barcode] += 1
    else:
        barcode_counts[barcode] = 1
        
print(barcode_counts)

{'ACTAAATAACTTAAGC': 63, 'AACCGTGACCAGGAAG': 70, 'CCACATTCATCGCTGA': 30, 'TTATCGTCTCCCATAT': 78, 'CTACCCGTTTCCCAAC': 124, 'CATACCAGTCATCCCT': 29, 'ACTTACGTATAAGTCA': 56, 'GCTACTACTATACCAT': 127, 'CGTCTTCCATCCCCAT': 136, 'GCGAGTCCACCAATTG': 97, 'GTTACCCACAGTCCGC': 64, 'ATATAAATAAGCTATG': 51, 'CACCACACAAGGATGT': 43, 'TCTTTTCTTATTGAGG': 74, 'CTTAACCTTCCGACAA': 57, 'ATAAACGTTCCCCAAG': 111, 'TCATACATCACACTTA': 48, 'GTACCCTCCGTGAATC': 100, 'ACTCCACGCTACCACG': 34, 'GCACTCCTCAACCCTT': 48, 'CCGCTCCCTGCTGTCC': 43, 'ACCCGTGAACCAAAGC': 1, 'AATCGTACATCCCCAC': 127, 'CCACCAGCCATACGGC': 93, 'AAACGTAGCGATAACT': 62, 'TGGACTCCACACCCCG': 79, 'TCACGTCCCATATTAC': 10, 'TCAAGAAGCCTTGGAG': 152, 'TTTCTCCCGCATCTCG': 105, 'AATCAATACGACCTGT': 74, 'TGACAATACCCTACTC': 48, 'CCCGACCCGACATTAA': 158, 'ACGAGAGGTCGACTCG': 61, 'TTCGACTTCCTAGTAC': 87, 'TTTCTCCCCCATCTCG': 1, 'CATTCACTACTCCCAA': 94, 'TCACCAGTGATCCCTT': 67, 'ACCACCGCCACACCCT': 47, 'ACCCAGTCTAGCTAAC': 71, 'GGTCATACGCCTTCGC': 17, 'GCTGCATTCCTGTGGC': 74, 'GTTCAGG